In [ ]:
import os
from pathlib import Path
import random
import time
import shutil
import zipfile

import kaggle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from ultralytics import YOLO

random_state = random.randint(1,999)
display(random_state)

# COnfiguraciones para tensorflow
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
! cat /root/.config/Ultralytics/settings.json
! sed -i 's|"datasets_dir":.*|"datasets_dir": "/app",|' /root/.config/Ultralytics/settings.json
! cat /root/.config/Ultralytics/settings.json

In [ ]:
# Download data from kaggle

RAW_DATA_ZIP = "data/raw/data2/rsud20k-bangladesh-road-scene-understanding.zip"
EXTRACTED_DATA = "data/preprocessed/data2"

dataset = "hasibzunair/rsud20k-bangladesh-road-scene-understanding"
kaggle.api.dataset_download_files(dataset, path='data/raw/data2')

with zipfile.ZipFile(RAW_DATA_ZIP, 'r') as zip_ref:
        zip_ref.extractall(EXTRACTED_DATA)
        print(f"Extracted all files to {EXTRACTED_DATA}")

DATA_DIR = Path("data/preprocessed/data2/rsud20k")
DATA_DIR

In [ ]:
def generate_and_validate_paths(data_dir):
    subdirs = ['images', 'labels']
    categories = ['test', 'train', 'val']
    folder_dict = {}

    print("Printing folder paths and checking their validity:")
    for subdir in subdirs:
        for category in categories:
            folder_name = f"{subdir}_{category}"
            folder_path = data_dir / subdir / category
            folder_dict[folder_name] = folder_path
            print(f"{folder_name}: {folder_path}")
            print(f"Is the path valid?: {folder_path.exists()}")

    return folder_dict

folder_dict = generate_and_validate_paths(DATA_DIR)
folder_dict

In [5]:
def generate_image_paths(data_dir):
    subdirs = ['images']
    data_dir = Path(data_dir)
    images_folder = data_dir / subdirs[0]
    categories = ['test', 'train', 'val']
    folder_paths = [images_folder / category for category in categories]
    train_folder = images_folder / 'train'
    train_images = list(train_folder.glob('*'))
    return folder_paths, train_images

def process_image_folders(folder_list, img_ext):
    img_name_list = []
    img_width_list = []
    img_height_list = []
    img_folder_list = []
    img_format_list = []

    for folder in folder_list:
        parent_name = folder.name
        for img_path in folder.iterdir():
            if img_path.is_file():
                img_folder_list.append(parent_name)
                img_name_list.append(img_path.name)

                img_format = img_path.suffix.lower()
                img_format_list.append(
                    "ok" if img_format in img_ext else "not ok")

                with Image.open(img_path) as img:
                    width, height = img.size
                    img_width_list.append(width)
                    img_height_list.append(height)

    data_model = {
        "folder": img_folder_list,
        "image_name": img_name_list,
        "width": img_width_list,
        "height": img_height_list,
        "format": img_format_list
    }

    return pd.DataFrame(data=data_model)

IMAGE_EXT = {'.jpg', '.jpeg', '.png'}
LABEL_EXT = {'.txt'}
folder_list, train_images = generate_image_paths(DATA_DIR)
report_df = process_image_folders(folder_list, IMAGE_EXT)

In [ ]:
report_df_group = report_df.groupby(
    ['folder', 'format', 'width', 'height']).count()
report_df_group

In [ ]:
# Shuffle train images
random.shuffle(train_images)
selected_images = train_images[:9]

print("Selected image paths after shuffle:")
for img in selected_images:
    print(img)

In [ ]:
def show_ima_matrix(image_list, show_axis=False):
    MAX_SIZE = 9
    if len(image_list) != MAX_SIZE:
        raise(f"Se requieren exactamente {MAX_SIZE} imágenes para crear la matriz.")

    plt.figure(figsize=(15, 10))
    gs1 = gridspec.GridSpec(3, 3)
    gs1.update(wspace=0.15, hspace=0.005)
    for i in range(MAX_SIZE):
        ima_file = image_list[i]
        ima_name = os.path.basename(ima_file)
        ax = plt.subplot(gs1[i])
        plt.imshow(np.array(Image.open(ima_file)))
        plt.title(ima_name[:20])
        plt.axis("on" if show_axis else "off")
    plt.show()
show_ima_matrix(selected_images, show_axis=True)

In [9]:
def verify_alignment(folder_dict):
    """
    Verifies alignment between images and corresponding label files.

    Args:
        folder_dict (dict): Dictionary containing image and label folder paths.

    Returns:
        None: Prints images without labels and labels without images for each folder.
    """
    categories = ['train', 'test', 'val']
    
    for category in categories:
        img_folder = folder_dict[f'images_{category}']
        lbl_folder = folder_dict[f'labels_{category}']
        
        img_files = {f.stem for f in img_folder.iterdir() if f.suffix.lower() in IMAGE_EXT}
        lbl_files = {f.stem for f in lbl_folder.iterdir() if f.suffix.lower() in LABEL_EXT}

        imgs_without_labels = img_files - lbl_files
        if imgs_without_labels:
            print(f"Imágenes sin etiquetas en la carpeta {img_folder}: {', '.join(imgs_without_labels)}")

        labels_without_imgs = lbl_files - img_files
        if labels_without_imgs:
            print(f"Etiquetas sin imágenes en la carpeta {lbl_folder}: {', '.join(labels_without_imgs)}")

verify_alignment(folder_dict)

In [54]:
def create_datasets(folder_dict, batch_size=4, image_size=(640, 640), sample_size=0.1):
    """
    Creates TensorFlow datasets for training, validation, and test with streaming data loading,
    and also samples a subset of the dataset into new folders.

    Args:
        folder_dict (dict): Dictionary containing folder paths for images and labels.
        batch_size (int): Size of the data batches for training, validation, and test.
        image_size (tuple): Size to which images will be resized (height, width).
        sample_size (float): Percentage of the dataset to sample and move to new folders.

    Returns:
        train_dataset (tf.data.Dataset): Streaming dataset for training.
        val_dataset (tf.data.Dataset): Streaming dataset for validation.
        test_dataset (tf.data.Dataset): Streaming dataset for test.
    """

    def create_image_and_label_paths(image_folder, label_folder):
        """
        Generates lists of image paths and corresponding label paths.

        Args:
            image_folder (Path): Folder containing images.
            label_folder (Path): Folder containing labels.

        Returns:
            image_paths (list): List of image paths.
            label_paths (list): List of corresponding label paths.
        """
        image_paths = sorted(list(image_folder.glob('*.jpg')))
        label_paths = [label_folder / img_path.with_suffix('.txt').name for img_path in image_paths]
        return image_paths, label_paths

    def load_image_and_label(image_path, label_path):
        """
        Loads an image and its corresponding label.

        Args:
            image_path (str): Path to the image.
            label_path (str): Path to the label.

        Returns:
            image (tensor): Resized and normalized image.
            label (tensor): Label as a tensor.
        """
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, image_size)
        image = image / 255.0

        label = tf.io.read_file(label_path)
        label = tf.strings.strip(label)
        label = tf.strings.split(label, '\n')

        label = tf.strings.split(label)
        label = tf.strings.to_number(label, out_type=tf.float32)

        return image, label

    def image_label_generator(image_paths, label_paths):
        """
        Generates image-label pairs in streaming mode.

        Args:
            image_paths (list): List of image paths.
            label_paths (list): List of corresponding label paths.

        Yields:
            image (tensor): Resized and normalized image.
            label (tensor): Label as a tensor.
        """
        for img_path, lbl_path in zip(image_paths, label_paths):
            yield str(img_path), str(lbl_path)  # Convert paths to strings

    def create_sampled_folders(sampled_image_paths, sampled_label_paths, sample_folder):
        """
        Creates new folders for sampled images and labels and moves files into them.

        Args:
            sampled_image_paths (list): List of sampled image paths.
            sampled_label_paths (list): List of corresponding sampled label paths.
            sample_folder (Path): Destination folder for the sampled data.
        """
        image_folder = sample_folder / "images"
        label_folder = sample_folder / "labels"
        os.makedirs(image_folder, exist_ok=True)
        os.makedirs(label_folder, exist_ok=True)

        for img_path, lbl_path in zip(sampled_image_paths, sampled_label_paths):
            # Load and transform the image
            image = tf.io.read_file(str(img_path))  # Ensure to convert to string
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, image_size)
            image = image / 255.0

            # Save the transformed image
            transformed_image_path = image_folder / img_path.name
            tf.io.write_file(str(transformed_image_path), tf.image.encode_jpeg(tf.cast(image * 255, tf.uint8)))

            # Copy the label file
            shutil.copy(str(lbl_path), label_folder / lbl_path.name)  # Ensure to convert to string

        print(f"Sampled images saved to: {image_folder}")
        print(f"Sampled labels saved to: {label_folder}")

    # Paths for train, validation, and test datasets
    train_image_paths, train_label_paths = create_image_and_label_paths(
        folder_dict['images_train'], folder_dict['labels_train']
    )
    
    val_image_paths, val_label_paths = create_image_and_label_paths(
        folder_dict['images_val'], folder_dict['labels_val']
    )
    
    test_image_paths, test_label_paths = create_image_and_label_paths(
        folder_dict['images_test'], folder_dict['labels_test']
    )

    # Sample: select a percentage of the training, validation, and test sets
    total_train_samples = len(train_image_paths)
    sample_size_count = int(total_train_samples * sample_size)
    
    sampled_train_image_paths = train_image_paths[:sample_size_count]
    sampled_train_label_paths = train_label_paths[:sample_size_count]

    # Create new sampled folders for train, val, and test
    sample_train_folder = Path("data/sample/train")
    sample_val_folder = Path("data/sample/val")
    sample_test_folder = Path("data/sample/test")

    create_sampled_folders(sampled_train_image_paths, sampled_train_label_paths, sample_train_folder)
    create_sampled_folders(val_image_paths, val_label_paths, sample_val_folder)
    create_sampled_folders(test_image_paths, test_label_paths, sample_test_folder)

    # Streaming datasets (for TensorFlow training)
    train_dataset = tf.data.Dataset.from_generator(
        lambda: image_label_generator(sampled_train_image_paths, sampled_train_label_paths),
        output_signature=(
            tf.TensorSpec(shape=(), dtype=tf.string),
            tf.TensorSpec(shape=(), dtype=tf.string)
        )
    )
    train_dataset = train_dataset.map(
        lambda img, lbl: load_image_and_label(img, lbl), num_parallel_calls=tf.data.AUTOTUNE
    )
    train_dataset = train_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_generator(
        lambda: image_label_generator(val_image_paths, val_label_paths),
        output_signature=(
            tf.TensorSpec(shape=(), dtype=tf.string),
            tf.TensorSpec(shape=(), dtype=tf.string)
        )
    )
    val_dataset = val_dataset.map(
        lambda img, lbl: load_image_and_label(img, lbl), num_parallel_calls=tf.data.AUTOTUNE
    )
    val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    test_dataset = tf.data.Dataset.from_generator(
        lambda: image_label_generator(test_image_paths, test_label_paths),
        output_signature=(
            tf.TensorSpec(shape=(), dtype=tf.string),
            tf.TensorSpec(shape=(), dtype=tf.string)
        )
    )
    test_dataset = test_dataset.map(
        lambda img, lbl: load_image_and_label(img, lbl), num_parallel_calls=tf.data.AUTOTUNE
    )
    test_dataset = test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_dataset, val_dataset, test_dataset

In [ ]:
train_dataset, val_dataset, test_dataset = create_datasets(
    folder_dict,
    image_size=(640, 640),
    sample_size=0.2,
    # sample_val_test=False
)

# Verify
for image, label in train_dataset.take(1):
    print("Image shape:", image.numpy().shape)
    print("Label:", label.numpy())

In [57]:
folder_dict_sample = {
    'images_test': Path('data/sample/test/images'),
    'images_train': Path('data/sample/train/images'),
    'images_val': Path('data/sample/val/images'),
    'labels_test': Path('data/sample/test/labels'),
    'labels_train': Path('data/sample/train/labels'),
    'labels_val': Path('data/sample/val/labels')
}
verify_alignment(folder_dict_sample)

In [ ]:
image_list2 = list(folder_dict_sample.get("images_train").glob("*.jpg"))[:9]
random.shuffle(image_list2)
print("Selected image paths after shuffle:")
for img in image_list2:
    print(img)
show_ima_matrix(image_list2, show_axis=True)

In [ ]:
yaml_content = f"""

train: sample/train/images
val: sample/val/images
test: sample/test/images

train_labels: sample/train/labels
val_labels: sample/val/labels
test_labels: sample/test/labels

# Clases que el modelo debe detectar
names:
  0: person
  1: rickshaw
  2: rickshaw van
  3: auto rickshaw
  4: truck
  5: pickup truck
  6: private car
  7: motorcycle
  8: bicycle
  9: bus
  10: micro bus
  11: covered van
  12: human hauler
"""

with open('data/settings_no_tuned.yaml', 'w') as file:
    file.write(yaml_content)

! cat data/settings_no_tuned.yaml

In [ ]:
YOLO_VERSION = 'yolov8m.pt'
model = YOLO(f'{YOLO_VERSION}')

start_time = time.time()

model.train(
    data='data/settings_no_tuned.yaml',
    epochs=15,
    imgsz=640,
    batch=4,
    lr0=0.01,
)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo total de entrenamiento: {elapsed_time:.2f} segundos")

metrics = model.val(data='data/settings_no_tuned.yaml')
print(metrics)

In [ ]:
yaml_content = f"""
# Ruta raíz de las imágenes y etiquetas
train: preprocessed/data2/rsud20k/images/train   # Ruta a las imágenes de entrenamiento
val: preprocessed/data2/rsud20k/images/val       # Ruta a las imágenes de validación
test: preprocessed/data2/rsud20k/images/test     # Ruta a las imágenes de prueba (opcional)

# Rutas a las etiquetas (opcional si se necesita)
train_labels: preprocessed/data2/rsud20k/labels/train   # Ruta a las etiquetas de entrenamiento
val_labels: preprocessed/data2/rsud20k/labels/val       # Ruta a las etiquetas de validación
test_labels: preprocessed/data2/rsud20k/labels/test     # Ruta a las etiquetas de prueba (opcional)

# Clases que el modelo debe detectar
names:
  0: person
  1: rickshaw
  2: rickshaw van
  3: auto rickshaw
  4: truck
  5: pickup truck
  6: private car
  7: motorcycle
  8: bicycle
  9: bus
  10: micro bus
  11: covered van
  12: human hauler

batch_size: 4
workers: 4
"""

with open('data/settings_tuned.yaml', 'w') as file:
    file.write(yaml_content)

! cat data/settings_tuned.yaml